In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time

In [2]:
# all opensea transactions
# those that execute atomicMatch_ function
# address from moralis endpoint /transactions/{transaction_hash}
# drop duplicates -> all collections
17000000/500*2/3600

18.88888888888889

In [3]:
# txn_hash, block_timestamp, block_number
marketplace_contract = "0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b"
api_key = "R8EERJ6PCoryCAkm2V4xHYakXBZ2K6Bloo5xdwO3MFtsy4prBIiL7IjYPAEBVscR"

In [4]:
def all_contract_transactions(marketplace_contract,api_key):
    headers = {"accept": "application/json", "X-API-Key":api_key}
    url =  f'https://deep-index.moralis.io/api/v2/{marketplace_contract}?chain=eth' # &offset=500&limit=1000
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        print(f"API returned status code {response.status_code}")
        print(f"Probably you have exceeded API limitations!")
        return
    content = response.json()
    limit = int(content["total"])
    total_pages = int(np.ceil(limit / 500))
    txn_hashes = []
    block_numbers = []
    block_timestamps = []
    for i in tqdm(range(total_pages)):
        try:
            offset = i*500
            url =  f'https://deep-index.moralis.io/api/v2/{marketplace_contract}?chain=eth&offset={offset}&limit={limit}'
            response = requests.request("GET", url, headers=headers)
            content = response.json()
            for tr in content["result"]:
                txn_hashes.append(tr["hash"])
                block_numbers.append(tr["block_number"])
                block_timestamps.append(tr["block_timestamp"])
        except:
            break
    collection_addreses = pd.DataFrame(np.array([txn_hashes,block_numbers,block_timestamps]).T, 
                                       columns=["txn_hash","block_number","block_timestamp"])
    collection_addreses["block_timestamp"] = pd.to_datetime(collection_addreses["block_timestamp"])
    return collection_addreses

In [ ]:
opensea_data = all_contract_transactions(marketplace_contract, api_key)

  0%|          | 0/33878 [00:00<?, ?it/s]

In [ ]:
opensea_data["block_timestamp"].min(),opensea_data["block_timestamp"].max()

In [ ]:
opensea_data.to_csv("/home/jmlago/opensea_data.csv")